In [ ]:
try:
  import pydrake
except ImportError:
  !curl -s https://raw.githubusercontent.com/RussTedrake/underactuated/master/scripts/setup/jupyter_setup.py > jupyter_setup.py
  from jupyter_setup import setup_drake
  setup_drake()
# Note: On Google's Colaboratory, this will take a minute, but should only need to reinstall once every 12 hours.
# Colab will ask you to "Reset all runtimes"; say no to save yourself the reinstall.

# MathematicalProgram Tutorial





## Background
Many engineering problems can be formulated as mathematical optimization problems, and solved by numerical solvers. A generic mathematical optimization problem can be formulated as
\begin{align}
\begin{array}{rl}
                       \min_x \;  &  f(x)
   \\[0.1pc]\text{subject to}  \;  &  x \in\mathcal{S}
   \end{array}
   \hspace{1.45em}
   {\scriptsize
   \begin{array}{|ll|}
             \hline \text{The real-valued decision variable is}      & x
       \\[-0.21pc]  \text{The real-valued cost function is}          & f(x)
       \\[-0.21pc]  \text{The constraint set is}                     & \mathcal{S}
       \\[-0.21pc]  \text{The optimal $x$ that minimizes the cost function is} & x^*
       \\[0.03pc]\hline
   \end{array}
   }
\end{align}

where $x$ is the real-valued decision variable(s), $f(x)$ is the real-valued *cost function*, $\mathcal{S}$ is the constraint set for $x$. Our goal is to find the optimal $x^*$ within the constraint set $\mathcal{S}$, such that $x^*$ minimizes the cost function $f(x)$.

For example, the following optimization problem determines the value of $x$ 
that minimizes $x^3 + 2x + 1$ subject to $x \ge 1$.
\begin{align}
\begin{array}{rl}
                       \min_x  &  x^3 + 2x + 1
   \\[0.1pc] \text{subject to}  &  x \ge 1
\end{array}
\hspace{1.45em}
{\scriptsize
\begin{array}{|ll|}
              \hline \text{The real-valued decision variable is}         &  x
   \\[-0.21pc]       \text{The real-valued cost function $f(x)$ is} &  x^3 + 2x + 1
   \\[-0.21pc]       \text{The set $\mathcal{S}$ of constraints is}      &  x \ge 1
   \\[-0.21pc]       \text{The value that minimizes the cost function is}    &  x^* = 1
   \\[0.03pc]\hline
\end{array}}
\end{align}

In general, how an optimization problem is solved depends on its categorization (categories include Linear Programming, Quadratic Programming, Mixed-integer Programming, etc.). Categorization depends on properties of both the cost function $f(x)$ and the constraint set $\mathcal{S}$. For example, if the cost function $f(x)$ is a linear function of $x$, and the constraint $\mathcal{S}$ is a linear set $\mathcal{S} = \{x | Ax\le b\}$, then we have a *linear programming* problem, which is efficiently solved with certain solvers. 

There are multiple solvers for each category of optimization problems,
but each solver has its own API and data structures.
Frequently, users need to rewrite code when they switch solvers.
To remedy this, Drake provides a common API through the *MathematicalProgram* class.
In addition to avoiding solver-specific code,
the constraint and cost functions can be written in symbolic form (which makes code more readable).
In these ways, Drake's MathematicalProgram is akin to [YALMIP](https://yalmip.github.io/) in MATLAB or [JuMP](https://github.com/JuliaOpt/JuMP.jl) in Julia, and we support both Python and C++.
<br> Note: Drake supports many [solvers](https://drake.mit.edu/doxygen_cxx/group__solvers.html)
(some are open-source and some require a license).

Drake can formulate and solve the following categories of optimization problems
* Linear programming
* Quadratic programming
* Second-order cone programming
* Nonlinear nonconvex programming
* Semidefinite programming
* Sum-of-squares programming
* Mixed-integer programming (mixed-integer linear programming, mixed-integer quadratic programming, mixed-integer second-order cone programming).
* Linear complementarity problem

This tutorial provides the basics of Drake's MathematicalProgram.
Advanced tutorials are available at
<a href='#advanced_tutorials'>link</a>.


## Basics of MathematicalProgram class
Drake's MathematicalProgram class contains the mathematical formulation of an optimization problem, namely the decision variables $x$, the cost function $f(x)$, and the constraint set $\mathcal{S}$.

### Initialize a MathematicalProgram object

 To initialize this class, first create an empty MathematicalProgram as

In [ ]:
from pydrake.solvers.mathematicalprogram import MathematicalProgram
import numpy as np

# Create an empty MathematicalProgram named prog (with no decision variables, 
# constraints or cost function)
prog = MathematicalProgram()



### Adding decision variables
Shown below, the function `NewContinuousVariables` adds two new continuous decision variables to `prog`.  The newly added variables are returned as `x` in a numpy array. 
<br><font size=-1> Note the range of the variable is a continuous set, as opposed to binary variables which only take discrete value 0 or 1.</font>

In [ ]:
x = prog.NewContinuousVariables(2)


The default names of the variable in *x* are "x(0)" and "x(1)".  The next line prints the default names and types in `x`, whereas the second line prints the symbolic expression "1 + 2x[0] + 3x[1] + 4x[1]".

In [ ]:
print(x)
print(1 + 2*x[0] + 3*x[1] + 4*x[1])

To create an array `y` of two variables named "dog(0)"" and "dog(1)", pass the name "dog" as a second argument to `NewContinuousVariables()`. Also shown below is the printout of the two variables in `y` and a symbolic expression involving `y`.

In [ ]:
y = prog.NewContinuousVariables(2, "dog")
print(y)
print(y[0] + y[0] + y[1] * y[1] * y[1])

To create a $3 \times 2$ matrix of variables named "A", type 

In [ ]:
var_matrix = prog.NewContinuousVariables(3, 2, "A")
print(var_matrix)

### Adding constraints
There are many ways to impose constraints on the decision variables. This tutorial shows a few simple examples. Refer to <a href='#advanced_tutorials'>advanced tutorials</a> for other types of constraints.



#### AddConstraint
The simplest way to add a constraint is with `MathematicalProgram.AddConstraint()`.

In [ ]:
# Add the constraint x(0) * x(1) = 1 to prog
prog.AddConstraint(x[0] * x[1] == 1)


You can also add inequality constraints to `prog` such as

In [ ]:
prog.AddConstraint(x[0] >= 0)
prog.AddConstraint(x[0] - x[1] <= 0)

`prog` automatically analyzes these symbolic inequality constraint expressions and determines they are all *linear* constraints on $x$.

### Adding Cost functions
In a complicated optimization problem, it is often convenient to write the total cost function $f(x)$ as a sum of individual cost functions
\begin{align}
f(x) = \sum_i g_i(x)
\end{align}


#### AddCost method.
The simplest way to add an individual cost function $g_i(x)$ to the total cost function $f(x)$ is with the `MathematicalProgram.AddCost()` method (as shown below).

In [ ]:
# Add a cost x(0)**2 + 3 to the total cost. Since prog doesn't have a cost before, now the total cost is x(0)**2 + 3
prog.AddCost(x[0] ** 2 + 3)

To add another individual cost function $x(0) + x(1)$ to the total cost function $f(x)$, simply call `AddCost()` again as follows

In [ ]:
prog.AddCost(x[0] + x[1])

now the total cost function becomes $x(0)^2 + x(0) + x(1) + 3$.

`prog` can analyze each of these individual cost functions and determine that $x(0) ^ 2 + 3$  is a convex quadratic function, and $x(0) + x(1)$ is a linear function of $x$.

### Solve the optimization problem
Once all the decision variables/constraints/costs are added to `prog`, we are ready to solve the optimization problem.



#### Automatically choosing a solver
The simplest way to solve the optimization problem is to call `Solve()` function. Drake's MathematicalProgram analyzes the type of the constraints/costs, and then calls an appropriate solver for your problem. The result of calling `Solve()` is stored inside the return argument. Here is a code snippet

In [ ]:
"""
Solves a simple optimization problem
       min x(0)^2 + x(1)^2
subject to x(0) + x(1) = 1
           x(0) <= x(1)
"""
from pydrake.solvers.mathematicalprogram import Solve
# Set up the optimization problem.
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(x[0] <= x[1])
prog.AddCost(x[0] **2 + x[1] ** 2)

# Now solve the optimization problem.
result = Solve(prog)

# print out the result.
print("Success? ", result.is_success())
# Print the solution to the decision variables.
print('x* = ', result.GetSolution(x))
# Print the optimal cost.
print('optimal cost = ', result.get_optimal_cost())
# Print the name of the solver that was called.
print('solver is: ', result.get_solver_id().name())

Notice that we can then retrieve optimization result from the return argument of `Solve`. For example, the solution $x^*$ is retrieved from `result.GetSolution()`, and the optimal cost from `result.get_optimal_cost()`.

Some optimization solution is infeasible (doesn't have a solution). For example in the following code example, `result.get_solution_result()` will not report `kSolutionFound`.

In [ ]:
"""
An infeasible optimization problem.
"""
prog = MathematicalProgram()
x = prog.NewContinuousVariables(1)[0]
y = prog.NewContinuousVariables(1)[0]
prog.AddConstraint(x + y >= 1)
prog.AddConstraint(x + y <= 0)
prog.AddCost(x)

result = Solve(prog)
print("Success? ", result.is_success())
print(result.get_solution_result())

#### Manually choosing a solver

If you want to choose a solver yourself, rather than Drake choosing one for you, you could instantiate a solver explicitly, and call its `Solve` function. For example, if I want to solve a problem using the open-source solver [IPOPT](https://github.com/coin-or/Ipopt), here is a code snippet

In [ ]:
"""
Demo on manually choosing a solver
Solves the problem
min x(0)
s.t x(0) + x(1) = 1
    0 <= x(1) <= 1
"""
from pydrake.solvers.ipopt import IpoptSolver
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddConstraint(x[0] + x[1] == 1)
prog.AddConstraint(0 <= x[1])
prog.AddConstraint(x[1] <= 1)
prog.AddCost(x[0])

# Choose IPOPT as the solver.
# First instantiate an IPOPT solver.

solver = IpoptSolver()
# The initial guess is [1, 1]. The third argument is the options for Ipopt solver,
# and we set no solver options.
result = solver.Solve(prog, np.array([1, 1]), None)

print(result.get_solution_result())
print("x* = ", result.GetSolution(x))
print("Solver is ", result.get_solver_id().name())
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())


Note that `solver.Solve()` expects three input arguments, the optimization program `prog`, the initial guess of the decision variable values (`[1, 1]` in this case), and an optional setting for the solver (`None` in this case, we use the default IPOPT setting). If you don't have an initial guess, you can substitute `np.array([1, 1])` with `None`. The solver will choose an initial guess automatically, as shown in the code below

In [ ]:
result = solver.Solve(prog, None, None)
print(result.get_solution_result())
print("x* = ", result.GetSolution(x))

Also note that if we know which solver is called, then we can access some solver-specific result, by calling `result.get_solver_details()`. For example, `IpoptSolverDetails` contains a field `status`, namely the status code of the IPOPT solver, we could access this info by

In [ ]:
print("Ipopt solver status: ", result.get_solver_details().status,
      ", meaning ", result.get_solver_details().ConvertStatusToString())

Each solver has its own details. You should refer to `XXXSolverDetails` class on what is stored inside the return argument of `result.get_solver_details()`. For example, if you know that IPOPT is called, then refer to `IpoptSolverDetails` class; for OSQP solver, refer to `OsqpSolverDetails`, etc.

### Setting solver options
When calling `Solve()` function, drake will use the default options for the solver (iterations, optimality tolerance, etc). You could modify these options in two ways, by either calling `MathematicalProgram::SetSolverOption`, or pass a `SolverOptions` argument to the `Solve()` function.



#### Calling MathematicalProgram::SetSolverOption
By calling `MathematicalProgram::SetSolverOption(solver_id, option_name, option_value)`, you can set a parameter for a specific solver (with the matching `solver_id`). The `option_name` is specific to that solver (for example, [here](https://www.coin-or.org/Ipopt/documentation/node40.html) is a list of IPOPT parameters). Note that `MathematicalProgram` object will store this solver parameter, and this parameter will be applied in the `Solve()` call, if that specific solver (with the matching `solver_id`) is invoked.

In the following code snippet, we show an example of setting the options of IPOPT.

In [ ]:
from pydrake.solvers.mathematicalprogram import SolverOptions
from pydrake.solvers.ipopt import IpoptSolver
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

# Set the maximum iteration for IPOPT to be 1.
# max_iter is a parameter of IPOPT solver, explained in
# https://www.coin-or.org/Ipopt/documentation/node42.html
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()
result = solver.Solve(prog, np.array([10, 1]), None)
# With fewer maximum iteration, IPOPT hasn't converged to optimality yet (The true optimal is [0.5, 0.5])
print("Success? ", result.is_success())
print(result.get_solution_result())
print("IPOPT x*= ", result.GetSolution(x))

Also note that setting the parameter of a solver **doesn't** mean that `result = Solve(prog)` will invoke that solver. The invoked solver is determined by Drake, to choose whichever solver it thinks most appropriate.

In the following snippet, although we set the solver options for IPOPT, Drake chooses another solver (which can solve this particular problem in the closed form.)

In [ ]:
prog.SetSolverOption(IpoptSolver().solver_id(), "max_iter", 1)
result = Solve(prog)
print(result.get_solver_id().name())

#### Passing a SolverOptions to Solve function
Another way of setting the solver options is to pass in a `SolverOptions` object as an argument to `Solve` function. `MathematicalProgram` will *not* store this `SolverOptions` object.

In the following example, in the first `Solve` call, it uses the `SolverOptions` object to set the parameter for IPOPT; in the second `Solve` call, it uses the default IPOPT parameters, hence we get different results from two `Solve` calls.

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2)
prog.AddCost(x[0]**2 + x[1] ** 2)
prog.AddConstraint(x[0] + x[1] == 1)

solver_options = SolverOptions()
solver_options.SetOption(IpoptSolver().solver_id(), "max_iter", 1)
solver = IpoptSolver()

# Call Solve with solver_options, IPOPT will use `max_iter` = 1
result = solver.Solve(prog, np.array([10, 1]), solver_options)
print("Success? ", result.is_success())
print(result.get_solution_result())
# Call Solve without solver_options, IPOPT will use the default options.
result = solver.Solve(prog, np.array([10, 1]), None)
print("Success? ", result.is_success())
print(result.get_solution_result())

## Advanced tutorials
<a id='advanced_tutorials'></a>

[Sum-of-squares optimization](https://drive.google.com/open?id=1vBkNigMecvz7lGEF_9jyD0FuzYY_iTTf)